# 加载库函数及导入文件

In [11]:
%load_ext autoreload
%autoreload 2
import warnings
import pathlib
from corelibs.config import *
warnings.filterwarnings('ignore', message="Workbook contains no default style, apply openpyxl's default", category=UserWarning)
conf_data = load_conf()
OUTPUT_DIR_PATH = pathlib.Path(r"J:\银行分析结果")
OUTPUT_DIR_PATH = pathlib.Path(r"D:\gitcodes\testdata\银行分析结果test")
OUTPUT_DIR_PATH = pathlib.Path(r"/mnt/d/gitcodes/testdata/银行分析结果test/")
OUTPUT_DIR_PATH

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


WindowsPath('/mnt/d/gitcodes/testdata/银行分析结果test')

In [ ]:
conf_data

# 执行

## 分析建设银行网点数据目录
（如：开发区金融街建行查询数据）

新版分析，可以根据账户信息更新账号数据：

In [21]:
from banks.ccb import process_dir_ccb_branch_v2
# OUTPUT_DIR_PATH = pathlib.Path(r"/mnt/d/gitcodes/testdata/银行分析结果test1/")
base_dir = pathlib.Path(r"D:\gitcodes\testdata\建行")
base_dir = pathlib.Path(r"/mnt/d/gitcodes/testdata/建行")
err_files = process_dir_ccb_branch_v2(base_dir, OUTPUT_DIR_PATH, 'a')

识别文件类型:   0%|          | 0/8 [00:00<?, ?it/s]

1080860401675667712130255_1_1_watermark.xlsx => 建设银行网点:账户
1080860401675667712130255_2_1_watermark.xlsx => 建设银行网点:流水:交易
1080860401675667712130255_3_1_watermark.xlsx => 建设银行网点:账户
1080860401675667712130255_4_1_watermark.xlsx => 建设银行网点:流水:交易
1080860401675667712130255_5_1_watermark.xlsx => 建设银行网点:账户
1080860401675667712130255_6_1_watermark.xlsx => 建设银行网点:流水:交易
1080860401675667712130255_7_1_watermark.xlsx => 建设银行网点:账户
1080860401675667712130255_8_1_watermark.xlsx => 建设银行网点:流水:交易


建设银行网点:账户:   0%|          | 0/4 [00:00<?, ?it/s]

1080860401675667712130255_1_1_watermark.xlsx……完成
1080860401675667712130255_3_1_watermark.xlsx……完成
1080860401675667712130255_5_1_watermark.xlsx……完成
1080860401675667712130255_7_1_watermark.xlsx……完成


建设银行网点:流水:   0%|          | 0/4 [00:00<?, ?it/s]

1080860401675667712130255_2_1_watermark.xlsx……完成
1080860401675667712130255_4_1_watermark.xlsx……完成
1080860401675667712130255_6_1_watermark.xlsx……完成
1080860401675667712130255_8_1_watermark.xlsx……完成
配置建设银行网点:['交易']暂不支持
共0个文件出错：


老版分析，不能根据账户信息更新账号数据：

In [16]:
from banks.ccb import process_dir_ccb_branch_v1
# OUTPUT_DIR_PATH = pathlib.Path(r"/mnt/d/gitcodes/testdata/银行分析结果test1/")
base_dir = pathlib.Path(r"D:\gitcodes\testdata\建行")
base_dir = pathlib.Path(r"/mnt/d/gitcodes/testdata/建行")
err_files = process_dir_ccb_branch_v1(base_dir, OUTPUT_DIR_PATH, 'a')

分析目录文件:   0%|          | 0/8 [00:00<?, ?it/s]

1080860401675667712130255_1_1_watermark.xlsx……建设银行网点:账户完成:
1080860401675667712130255_2_1_watermark.xlsx……建设银行网点:流水完成:交易暂不支持:
1080860401675667712130255_3_1_watermark.xlsx……建设银行网点:账户完成:
1080860401675667712130255_4_1_watermark.xlsx……建设银行网点:流水完成:交易暂不支持:
1080860401675667712130255_5_1_watermark.xlsx……建设银行网点:账户完成:
1080860401675667712130255_6_1_watermark.xlsx……建设银行网点:流水完成:交易暂不支持:
1080860401675667712130255_7_1_watermark.xlsx……建设银行网点:账户完成:
1080860401675667712130255_8_1_watermark.xlsx……建设银行网点:流水完成:交易暂不支持:
4个文件出错：
1080860401675667712130255_2_1_watermark.xlsx => 建设银行网点交易暂不支持
1080860401675667712130255_4_1_watermark.xlsx => 建设银行网点交易暂不支持
1080860401675667712130255_6_1_watermark.xlsx => 建设银行网点交易暂不支持
1080860401675667712130255_8_1_watermark.xlsx => 建设银行网点交易暂不支持


# 测试

## 提取excel表头和md5值

In [8]:
from hashlib import md5
from corelibs.header import read_header
acc_file = pathlib.Path(r"J:\杨孟军\银行\工商银行\1\陶瑞兰120221196603113061\1.xls")
acc_file = pathlib.Path(r"/mnt/d/gitcodes/testdata/000615/王瑞霞_410527198607032042/工商银行-账户信息-客户基本信息.xlsx")
base_dir = pathlib.Path(r"/mnt/d/gitcodes/testdata/天津/1.xls")
print(a := read_header(acc_file, header=0))
md5(a.encode()).hexdigest()

('联系手机', '单位地址', '客户名称', '地税纳税号', '客户工商执照号码', '代办人姓名', '法人代表证件号码', '邮箱地址', '法人代表证件类型', '代办人证件类型', '工作单位', '账单地址', '法人代表', '住宅电话', '证照号码', '联系电话', '单位电话', '国税纳税号', '证照类型代码', '代办人证件号码', '住宅地址')


'5a4e032332e1e4fe1cb3f0843c893966'

## 测试yaml配置文件语法，及内部操作指令对象转换

In [7]:
load_conf()
print(get_output_format())
print(len([print(x) for x in get_header_hash().items()]))

excel
('6d3d0d1e252f43a000010e19d8b2dd57', ['天津银行', '账户'])
('7509cd85808831c7004c8c78e9267d7c', ['天津银行', '流水'])
('5b70f468037b3cfe76bace72751f4e3e', ['工商银行网点', '账户'])
('0b02a413bd3c59d74a82e3fd3909c759', ['工商银行网点', '流水'])
('5ff3970075646f3a7fffdef5430539d7', ['建设银行网点', '账户'])
('8fa9c0776b857793f91335192398aeba', ['建设银行网点', '流水', '交易'])
6


In [ ]:
print(len([print(x) for x in get_conf_cache().items()]))

In [39]:
load_conf()
conf_obj = get_conf_obj('天津银行','流水')
[print(x) for x in conf_obj]

None
{'银行': '天津银行'}
{'我方中文名称': True, '我行账号（卡号）': True, '余额': True, '借贷标志': True, '交易金额': True, '交易日期': True}
{'我方中文名称': '姓名', '我行账号（卡号）': '账号', '用途（摘要）': '摘要', '对方中文名称': '对方户名', '对方账号（卡号）': '对方账号', '对方银行名称': '对方开户行', '网银IP地址': 'IP/MAC地址'}
{'交易机构': ['受理机构名称', '我方银行名称']}
{'日期': ['交易日期']}
{'时间': ['交易日期']}
{'余额': True, '交易金额': True}
{'C': '借', 'CD_col': '借贷标志', 'C_col': '出账金额', 'trans_col': '交易金额', 'D': '贷', 'D_col': '入账金额'}
{}
['银行', '日期', '时间', '姓名', '账号', '卡号', '摘要', '币种', '出账金额', '入账金额', '余额', '交易机构', '对方户名', '对方账号', '对方开户行', 'IP/MAC地址', '借贷标志', '交易金额', '交易日期', '受理机构名称', '我方银行名称', '对方银行机构代码', '受理机构代码', '交易日期', '借贷标志', '交易金额', '身份证号', '流水号', '交易代码（交易渠道）', '代码描述', '交易是否成功']
{'卡号': ['账号', '卡号', '账号']}


[None, None, None, None, None, None, None, None, None, None, None, None]

##  测试单个账户文件读取及储存

In [33]:
from corelibs.process import process_account_file_general
acc_file = pathlib.Path(r"E:\张楠\2023赵益华927\银行\建行\第一批 赵益华等9人\1080861461690963794022777_1_1赵益华.xlsx")
# acc_file = pathlib.Path(r"E:\张楠\2023赵益华927\银行\天津银行\滨监查询（2023）05217号.xls")
# acc_file = pathlib.Path(r"/mnt/d/gitcodes/testdata/建行(于东林、刘德阳等)【已分析】/1080860401675667712130255_5_1_watermark.xlsx")
df_acc = process_account_file_general(acc_file, OUTPUT_DIR_PATH,'建设银行网点', '账户')
# df_acc

In [31]:
import dtale

In [35]:
dtale.show(df_acc, open_browser=True)

## 测试单个流水文件的读取与储存

In [43]:
from corelibs.process import process_statment_file_general
acc_file = pathlib.Path(r"E:\张楠\2023赵益华927\银行\建行\第一批 赵益华等9人\1080861461690963794022777_2_1_watermark.xlsx")
# acc_file = pathlib.Path(r"E:\张楠\2023赵益华927\银行\天津银行\滨监查询（2023）05217号.xls")
# acc_file = pathlib.Path(r"/mnt/d/gitcodes/testdata/建行(于东林、刘德阳等)【已分析】/1080860401675667712130255_6_1_watermark.xlsx")
df = process_statment_file_general(acc_file, OUTPUT_DIR_PATH,'建设银行网点', '流水','a', df_acc)

In [ ]:
import dtale

In [44]:
dtale.show(df, open_browser=True)

## 测试识别文件列表中的文件类型

In [18]:
from collections import Counter
from corelibs.process import classify_files_by_category

base_dir = pathlib.Path(r"D:\gitcodes\testdata\建行")
base_dir = pathlib.Path(r"/mnt/d/gitcodes/testdata/建行/")
_file_names = list(base_dir.glob('[!~]*.xlsx')) # 找到目录中所有的excel文件（不含子目录）
f,e = classify_files_by_category(_file_names)

print(f)
len([print(k.name + v) for k, v in e.items()])

识别文件类型:   0%|          | 0/8 [00:00<?, ?it/s]

1080860401675667712130255_1_1_watermark.xlsx => 建设银行网点:账户
1080860401675667712130255_2_1_watermark.xlsx => 建设银行网点:流水:交易
1080860401675667712130255_3_1_watermark.xlsx => 建设银行网点:账户
1080860401675667712130255_4_1_watermark.xlsx => 建设银行网点:流水:交易
1080860401675667712130255_5_1_watermark.xlsx => 建设银行网点:账户
1080860401675667712130255_6_1_watermark.xlsx => 建设银行网点:流水:交易
1080860401675667712130255_7_1_watermark.xlsx => 建设银行网点:账户
1080860401675667712130255_8_1_watermark.xlsx => 建设银行网点:流水:交易
{'建设银行网点': {'账户': [PosixPath('/mnt/d/gitcodes/testdata/建行/1080860401675667712130255_1_1_watermark.xlsx'), PosixPath('/mnt/d/gitcodes/testdata/建行/1080860401675667712130255_3_1_watermark.xlsx'), PosixPath('/mnt/d/gitcodes/testdata/建行/1080860401675667712130255_5_1_watermark.xlsx'), PosixPath('/mnt/d/gitcodes/testdata/建行/1080860401675667712130255_7_1_watermark.xlsx')], '流水': [PosixPath('/mnt/d/gitcodes/testdata/建行/1080860401675667712130255_2_1_watermark.xlsx'), PosixPath('/mnt/d/gitcodes/testdata/建行/10808604016756677121302

0

In [ ]:
base_dir = pathlib.Path(r'D:\gitcodes\testdata\建行')
list(base_dir.glob('[!~]*.xlsx')) 


[]

# 杂项

In [28]:
import pandas as pd
df2 = pd.DataFrame({'a':[1,2,3,4],'b':[5,6,7,8],'c':[9,10,11,12]})
df1 = pd.DataFrame({'a':[1,2,1,5],'b':[1,2,1,5],'d':[13,14,15,16]})
print(df1)
print(df2)
a = pd.merge(df1,df2,left_on='a',right_on='a', how='left', validate='m:1', copy=False)
a['c'].where(a['c'].notnull(), a['d'], inplace=True)
a

   a  b   d
0  1  1  13
1  2  2  14
2  1  1  15
3  5  5  16
   a  b   c
0  1  5   9
1  2  6  10
2  3  7  11
3  4  8  12


,a,b_x,d,b_y,c
0,1,1,13,5.0,9.0
1,2,2,14,6.0,10.0
2,1,1,15,5.0,9.0
3,5,5,16,NaN,16.0
